In [1]:
import numpy as np
import pandas as pd
from ast import literal_eval
from pathlib import Path
from pprint import pprint
from sympy.combinatorics import Permutation
import matplotlib.pyplot as plt

# local data_dir
data_dir = Path("data")

puzzle_info = pd.read_csv(data_dir / 'puzzle_info.csv', index_col='puzzle_type')
# Parse allowed_moves
puzzle_info['allowed_moves'] = puzzle_info['allowed_moves'].apply(literal_eval)
# puzzle_info_df['total_allowed_moves'] = puzzle_info_df['allowed_moves_dict'].apply(lambda x: len(x.keys()))
puzzle_info['number_moves'] = puzzle_info['allowed_moves'].apply(lambda x: len(x.keys()))

puzzles_all = pd.read_csv(data_dir / 'puzzles.csv', index_col='id')
# Parse color states
puzzles_all = puzzles_all.assign(
    initial_state=lambda df: df['initial_state'].str.split(';'),
    solution_state=lambda df: df['solution_state'].str.split(';')
)

puzzles_all['total_components'] = puzzles_all['solution_state'].apply(len)
puzzles_all['all_unique_components'] = puzzles_all['solution_state'].apply(lambda x: np.unique(x))
puzzles_all['unique_components'] = puzzles_all['all_unique_components'].apply(len)

# load the sample_submission
ss = pd.read_csv(data_dir / 'sample_submission.csv', index_col='id')
ss['moves'] = ss['moves'].str.split('.')
ss['move_count'] = ss['moves'].apply(len)


In [2]:
# work with puzzle 30
puzzle_id = 30
puzzle = puzzles_all.loc[puzzle_id]

# extract initial state and final state as np.array
initial_state = np.array(puzzle['initial_state'])
final_state = np.array(puzzle['solution_state'])

# create ditionary of allowed moves
allowed_moves = puzzle_info.loc[puzzle['puzzle_type']]['allowed_moves']
# create dictionary of allowed moves converting list to np.array
allowed_moves = {k: np.array(v) for k, v in allowed_moves.items()}
# include inverse moves
move_key = list(allowed_moves.keys())
for i in range(len(allowed_moves)):
    move = allowed_moves[move_key[i]]
    allowed_moves[f"-{move_key[i]}"] = np.argsort(move)


# create reverse_states_df with distance column == 0 and state column == final state
reverse_states_df = pd.DataFrame(columns=['distance', 'state'])
reverse_states_df.loc[0] = [0, final_state]
used_states = set(final_state)
i = 1
for k, v in allowed_moves.items():
    new_state = final_state[v]
    if tuple(new_state) not in used_states:
        l = len(reverse_states_df)
        # add info to reverse_states_df
        reverse_states_df.loc[l] = [i, new_state]
        # add new_state to used_states
        used_states.add(tuple(new_state))



    

In [47]:
for i in range(0, 100):
    print(f"id {i}: type: {puzzles_all.loc[i]['puzzle_type']}, unique_components: {puzzles_all.loc[i]['unique_components']}, moves: {ss.loc[i]['move_count']}")


id 0: type: cube_2/2/2, unique_components: 6, moves: 2
id 1: type: cube_2/2/2, unique_components: 6, moves: 63
id 2: type: cube_2/2/2, unique_components: 6, moves: 62
id 3: type: cube_2/2/2, unique_components: 6, moves: 92
id 4: type: cube_2/2/2, unique_components: 6, moves: 70
id 5: type: cube_2/2/2, unique_components: 6, moves: 54
id 6: type: cube_2/2/2, unique_components: 6, moves: 68
id 7: type: cube_2/2/2, unique_components: 6, moves: 83
id 8: type: cube_2/2/2, unique_components: 6, moves: 98
id 9: type: cube_2/2/2, unique_components: 6, moves: 76
id 10: type: cube_2/2/2, unique_components: 6, moves: 66
id 11: type: cube_2/2/2, unique_components: 6, moves: 63
id 12: type: cube_2/2/2, unique_components: 6, moves: 72
id 13: type: cube_2/2/2, unique_components: 6, moves: 131
id 14: type: cube_2/2/2, unique_components: 6, moves: 96
id 15: type: cube_2/2/2, unique_components: 6, moves: 68
id 16: type: cube_2/2/2, unique_components: 6, moves: 63
id 17: type: cube_2/2/2, unique_component

In [4]:
# extract the state column from reverse_states_df and store it as a np.array
reverse_states = np.array(reverse_states_df['state'].tolist())
reverse_states.shape

# perform row-wise comparison of reverse_states with final_state and sum the number of matches
np.sum(reverse_states != final_state, axis=1)

array([ 0, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12])

In [3]:
# work with puzzle 30
puzzle_id = 30
puzzle = puzzles_all.loc[puzzle_id]

# extract initial state and final state as np.array
initial_state = np.array(puzzle['initial_state'])
final_state = np.array(puzzle['solution_state'])

# create ditionary of allowed moves
allowed_moves = puzzle_info.loc[puzzle['puzzle_type']]['allowed_moves']
# create dictionary of allowed moves converting list to np.array
allowed_moves = {k: np.array(v) for k, v in allowed_moves.items()}
# include inverse moves
move_key = list(allowed_moves.keys())
for i in range(len(allowed_moves)):
    move = allowed_moves[move_key[i]]
    allowed_moves[f"-{move_key[i]}"] = np.argsort(move)

# create reverse_states_df with distance column == 0 and state column == final state
reverse_states_df = pd.DataFrame(columns=['distance', 'state'])
reverse_states_df.loc[0] = [0, final_state]
used_states = set(final_state)

max_depth = 5  # Define the maximum depth of the search
while reverse_states_df['distance'].max() < max_depth:
    current_depth = reverse_states_df['distance'].max()
    states_to_expand = reverse_states_df[reverse_states_df['distance'] == current_depth]

    for index, row in states_to_expand.iterrows():
        current_state = row['state']
        for move_name, move in allowed_moves.items():
            new_state = current_state[move]
            if tuple(new_state) not in used_states:
                reverse_states_df.loc[len(reverse_states_df)] = [current_depth + 1, new_state]
                used_states.add(tuple(new_state))


KeyboardInterrupt: 

In [4]:
# create dictionary of reverse states
state_distance_dict = {tuple(row['state']): row['distance'] for _, row in reverse_states_df.iterrows()}



In [5]:
len(state_distance_dict)

174198

In [6]:
import heapq
import time

def base_heuristic(intermediate_state, final_state):
    """
    Calculate a base heuristic for the given state.
    This is just a placeholder function; you should replace it with your actual heuristic logic.
    """
    # Example heuristic logic (to be replaced with your specific heuristic calculation)
    return np.sum(np.abs(intermediate_state - final_state))

def get_state_distance(intermediate_state, state_distance_dict, final_state):
    """
    Return the distance of the intermediate state from the final state.
    If the state is not found, revert to the base heuristic.
    """
    return state_distance_dict.get(tuple(intermediate_state), evaluate_difference(intermediate_state, final_state))

def evaluate_difference(state1, state2):
    """
    Count the number of positions where the values in two numpy arrays differ.

    Args:
    state1 (np.array): First state, a numpy array.
    state2 (np.array): Second state, a numpy array.

    Returns:
    int: The number of differing positions.
    """
    if state1.shape != state2.shape:
        raise ValueError("States must have the same shape.")

    # Count the number of differing elements
    return np.sum(state1 != state2)

def astar(move_dict, initial_state, final_state, state_distance_dict, 
          wildcards, max_depth=50, timeout = 180, verbose=False):
    '''
    A* search algorithm. Returns the path of moves to solve the puzzle.
    Inputs:
        move_dict: Dictionary of moves
        initial_state: Initial state of the puzzle
        final_state: Final state of the puzzle
        state_distance_dict: Dictionary of states and their distances from the final state
        wildcards: Number of wildcards allowed
        max_depth: Maximum depth to search
        timeout: Maximum time to search
    Outputs:
        path: List of moves to solve the puzzle
        node_counter: Number of nodes visited
        time_delta: Time elapsed
        '''
    
    # Priority queue to store nodes with their f-values (g + h)
    start_time = time.time()
    open_set = []
    node_counter = 1

    final_state_np = np.array(final_state)
    
    # Use a heap to store nodes with the lowest f-value at the top in open_set
    # heap is a list of tuples (f-value, state, path)
    heapq.heappush(open_set, (0, initial_state, [])) 
    
    # create closed set to store visited nodes
    closed_set = set()

    while open_set:
        # Check for timeout
        time_delta = time.time() - start_time
        if time_delta > timeout:
            print("Timed out.")
            return None, node_counter, time_delta
        
        # Get the node with the lowest f-value from the heap open_set
        current_f, current_state, current_path = heapq.heappop(open_set)
        current_state_np = np.array(current_state)
        
        # Check if we've reached the max depth (early termination)
        if len(current_path) > max_depth:
            print("Max depth exceeded.")
            return None, node_counter, time_delta

        # Check if we've reached the goal and return the path, node_counter, and time_delta
        if evaluate_difference(current_state_np, final_state_np) <= wildcards:
            # We've achieved our goal. Return the move path.
            return current_path, node_counter, time_delta

        # Add the current state to the closed set
        closed_set.add(tuple(current_state))
        
        # Add the next possible moves to the open set
        for move_str, move in move_dict.items():
            # apply the move permutation to the current state
            new_state_np = current_state_np[move]
            new_state = list(new_state_np)
            if tuple(new_state) not in closed_set:
                if len(current_path) < max_depth:
                    # Add the new state to the open set tuple (f-value, state, path)
                    # f-value = g + h
                        # g is the number of moves taken so far plus 1
                        # h is the heuristic score of the new state using evaluate_score
                    # state is the new state
                    # path is the current path plus the new move

                    # # print commands
                    # print(f"node_counter: {node_counter}")
                    # print(f"len(current_path): {len(current_path)}")
                    # print(f"get_state_distance(): {get_state_distance(new_state, state_distance_dict, final_state)}")
                    # print(f"new_state: {new_state}")
                    # print(f"current_path: {current_path}")
                    if verbose and node_counter % 1000 == 0:
                        print(f"current_f: {current_f} and node_counter: {node_counter}")
                    heapq.heappush(open_set, 
                                   (len(current_path) + 1 + get_state_distance(new_state_np, state_distance_dict, final_state_np), 
                                    new_state,
                                    current_path + [move_str]))
                                    
                    # Increment the node counter
                    node_counter += 1
    
    # end of the while loop
    # If no solutions are found:
    print("Open set completed. No solutions.")
    time_delta = time.time() - start_time
    return None, node_counter, time_delta

def plot_final_path(move_dict, initial_state, final_state, path):
    state_list = [initial_state]
    for move in path:
        p = move_dict.get(move)
        state_list.append(p(state_list[-1]))
    eval_list = [get_state_distance(state, state_distance_dict, final_state) for state in state_list]
    plt.plot(eval_list)
    plt.xlabel("Move Number")
    plt.ylabel("Heuristic Score")
    plt.show()

In [7]:
for i in range(31, 35):
    # work with puzzle i
    puzzle_id = i
    puzzle = puzzles_all.loc[puzzle_id]

    # # extract initial state and final state as np.array
    # initial_state = np.array(puzzle['initial_state'])
    # final_state = np.array(puzzle['solution_state'])

    initial_state = puzzle['initial_state']
    final_state = puzzle['solution_state']

    path, nodes, time_delta = astar(allowed_moves, initial_state, final_state, state_distance_dict, 
            wildcards=0, max_depth=50, timeout = 180)
    if path is not None:
        print('SOLVED!')
        print(f"puzzle_id: {puzzle_id}\n\tnodes: {nodes}\n\ttime: {time_delta}\n\tpath: {path}")
    else:
        print('FAIL!!!')
        print(f"puzzle_id: {puzzle_id}\n\tnodes: {nodes}\n\ttime: {time_delta}")

Timed out.
FAIL!!!
puzzle_id: 31
	nodes: 6238447
	time: 183.5936119556427
Timed out.
FAIL!!!
puzzle_id: 32
	nodes: 6756381
	time: 180.00127601623535
Timed out.
FAIL!!!
puzzle_id: 33
	nodes: 6393048
	time: 185.97042107582092
Timed out.
FAIL!!!
puzzle_id: 34
	nodes: 6791848
	time: 180.00024700164795
